# Arrow detection using tensorflow

## 1. Create folder hierarchy

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pickle

files = pickle.load(open('files.pickle', 'rb'))
paths = pickle.load(open('paths.pickle', 'rb'))

## 2. Create Label Map

In [ ]:
labels = [{'name':'1', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## 3. Prepere test and train data

In [ ]:
import glob

#DATA='/home/ox330/data/targets/all/'
#DATA='/home/ox330/data/targets/squares/'
DATA='/home/ox330/data/targets/points/'

annotation_jsons = glob.glob(DATA + '*.json')
test_size = int(len(annotation_jsons) / 3)
#test_size = int(len(annotation_jsons) - 10)
test = annotation_jsons[:test_size]
train = annotation_jsons[test_size:]

!mkdir -p {os.path.join(paths['IMAGE_PATH'], 'test')}
!mkdir -p {os.path.join(paths['IMAGE_PATH'], 'train')}

!rm {os.path.join(paths['IMAGE_PATH'], 'test')}/*
!rm {os.path.join(paths['IMAGE_PATH'], 'train')}/*

for test1 in test:
    !cp {test1} {os.path.join(paths['IMAGE_PATH'], 'test')}
    !cp {os.path.splitext(test1)[0]+'.jpg'} {os.path.join(paths['IMAGE_PATH'], 'test')}
for train1 in train:
    !cp {train1} {os.path.join(paths['IMAGE_PATH'], 'train')}
    !cp {os.path.splitext(train1)[0]+'.jpg'} {os.path.join(paths['IMAGE_PATH'], 'train')}

## 4. tfrecord

In [ ]:
import generate_tfrecord as gen;
gen.json_to_tfrecord(os.path.join(paths['IMAGE_PATH'], 'train'), files['LABELMAP'],
    os.path.join(paths['IMAGE_PATH'], 'train'),
    os.path.join(paths['ANNOTATION_PATH'], 'train.record'), 40)
gen.json_to_tfrecord(os.path.join(paths['IMAGE_PATH'], 'test'), files['LABELMAP'],
    os.path.join(paths['IMAGE_PATH'], 'test'),
    os.path.join(paths['ANNOTATION_PATH'], 'test.record'), 40)

## 5. Copy Model Config to Training Folder

In [ ]:
!cp {files['PRETRAINED_MODEL_CONFIG']} {paths['CHECKPOINT_PATH']}

## 6. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
# config from pretrained model
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    text_format.Merge(f.read(), pipeline_config)

In [ ]:
# update model training parameters
pipeline_config.model.ssd.num_classes = len(labels)
# pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.height = 512
# pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.width = 512

pipeline_config.train_config.batch_size = 2
pipeline_config.train_config.fine_tune_checkpoint = files['PRETRAINED_MODEL_CKP0']
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

pipeline_config.train_input_reader.label_map_path = files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'train.record')]

pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
#store updated config
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

## 7. Train the Model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(
    TRAINING_SCRIPT,
    paths['CHECKPOINT_PATH'],
    files['PIPELINE_CONFIG'])

In [ ]:
!rm {paths['CHECKPOINT_PATH']}/ckpt-*
!rm {paths['CHECKPOINT_PATH']}/checkpoint

In [ ]:
print(command)

In [ ]:
!{command}

## 8. Evaluate the Model

In [ ]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(
    TRAINING_SCRIPT,
    paths['CHECKPOINT_PATH'],
    files['PIPELINE_CONFIG'],
    paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}